In [ ]:
##safety executable check (for conda env)
import sys
sys.executable

#### Working with Environment Sound Classification Dataset (https://github.com/karoldvl/ESC-50)
Testing feature extraction and several classifiers with the ECS-10 data set

#### Feature extraction

In [3]:
# coding= UTF-8
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
import soundfile as sf

##Return audio features 
def feature_extraction(file_name):
    X, sample_rate = librosa.load(file_name)
    if X.ndim > 1:
        X = X[:,0]
    X = X.T
    
    # Get features   
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0) #40 values
    #zcr = np.mean(librosa.feature.zero_crossing_rate)
    chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
    contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T, axis=0)
    tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T, axis=0) #tonal centroid features
    
    ##Return computed features
    return mfccs, chroma, mel, contrast, tonnetz
    
# Process audio files: Return arrays with features and labels
def parse_audio_files(parent_dir, sub_dirs, file_ext='*.ogg'): ## .ogg audio format
    features, labels = np.empty((0,193)), np.empty(0) # 193 features total. This can vary
    
    for label, sub_dir in enumerate(sub_dirs): ##The enumerate() function adds a counter to an iterable.
        for file_name in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)): ##parent is data, sub_dirs are the classes
            try:
                mfccs, chroma, mel, contrast, tonnetz = feature_extraction(file_name)
            except Exception as e:
                print("[Error] there was an error in feature extraction. %s" % (e))
                continue
                
            extracted_features = np.hstack([mfccs,chroma, mel, contrast, tonnetz]) #Stack arrays in sequence horizontally (column wise)
            features = np.vstack([features, extracted_features]) #Stack arrays in sequence vertically (row wise).
            labels = np.append(labels, label)
        print("Extracted features from %s, done" % (sub_dir))
    return np.array(features), np.array(labels, dtype = np.int) ## arrays with features and corresponding labels for each audio

def one_hot_encode(labels): ##Check this hot encode
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels, n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

# Read sub-directories (audio classes)
audio_directories = os.listdir("audio-data/")
audio_directories.sort()
print('Audio Classes: ', audio_directories)

('Audio Classes: ', ['001 - Dog bark', '002 - Rain', '003 - Sea waves', '004 - Baby cry', '005 - Clock tick', '006 - Person sneeze', '007 - Helicopter', '008 - Chainsaw', '009 - Rooster', '010 - Fire crackling'])


In [7]:
##Get labels and features
features, labels = parse_audio_files('audio-data', audio_directories) #(parent dir,sub dirs)
np.save('feat.npy', features) ##NumPy array file created. Files are binary files to store numpy arrays
np.save('label.npy', labels)

Extracted features from 001 - Dog bark, done
Extracted features from 002 - Rain, done
Extracted features from 003 - Sea waves, done
Extracted features from 004 - Baby cry, done
Extracted features from 005 - Clock tick, done


/home/hackerman/anaconda2/envs/anaconda2_py27/lib/python2.7/site-packages/librosa/core/pitch.py:145: UserWarning: Trying to estimate tuning from empty frequency set.
  warnings.warn('Trying to estimate tuning from empty frequency set.')


Extracted features from 006 - Person sneeze, done
Extracted features from 007 - Helicopter, done
Extracted features from 008 - Chainsaw, done
Extracted features from 009 - Rooster, done
Extracted features from 010 - Fire crackling, done


In [8]:
# Label integer encoding 
labels = np.load('label.npy') # 10 labels total
#print(labels)

# For future label de-encoding
label_classes = np.array(['Dog bark','Rain','Sea waves','Baby cry','Clock tick','Person sneeze','Helicopter','Chainsaw','Rooster',
                          'Fire crackling'])
print(label_classes)

['Dog bark' 'Rain' 'Sea waves' 'Baby cry' 'Clock tick' 'Person sneeze'
 'Helicopter' 'Chainsaw' 'Rooster' 'Fire crackling']


In [9]:
features= np.load('feat.npy')
print(len(features)) # 400 features total
print(features)

400
[[-3.12936138e+02  1.53756654e+02 -8.42777452e+01 ...  5.63282588e-02
   2.08516431e-02 -1.87674272e-02]
 [-3.20112853e+02  1.42200505e+02 -4.77227058e+01 ...  7.24356147e-02
   1.58665382e-02 -2.03768118e-03]
 [-5.69369487e+02  3.15002805e+01  9.34787663e-01 ... -9.74065657e-03
  -2.01712929e-03  2.35111669e-02]
 ...
 [-2.86730051e+02  6.26902425e+01  1.85678427e+01 ...  9.72864029e-03
   8.37429690e-03  1.61102826e-02]
 [-3.26840023e+02  9.06312882e+01  7.19092342e+01 ...  5.48059287e-02
   1.16528212e-02  9.53665151e-03]
 [-2.96461065e+02  7.54222837e+01  1.30356668e+01 ...  9.11465921e-02
   2.24434430e-02  1.89593115e-02]]


##### Data visualization

In [17]:
#Pandas dataframe with 193 features variables for each audio
df = pd.DataFrame(features)

# Add a new column for class (label), this is our target
df['Audio class'] = pd.Categorical.from_codes(labels, label_classes)

df[[0,1,2,3,4,5,6,7,8,9,10,11,'Audio class']]

0           1          2          3          4          5  \
0   -359.907670  176.984312 -28.405258 -77.346419 -23.394936  -8.336801   
1   -364.606377  160.027782 -10.176146 -49.205103 -47.292736 -24.890667   
2   -569.158646   30.929713   5.359668   0.996636   1.891372   2.343744   
3   -419.991445  158.331816  -5.460790  -2.632195  10.701890  13.474268   
4   -331.014025  124.460048 -20.819073 -29.484597 -36.932396  -7.367482   
5   -393.339552   78.339811   4.843938  -7.676779 -12.608208  -4.106547   
6   -387.847069  112.019940  11.978814 -11.534052 -15.415533   0.335486   
7   -430.738218   94.625625  11.179736  -7.445123  -2.928210  -3.732962   
8   -466.842492   57.002892 -22.152894 -25.998953 -18.681116  -5.563477   
9   -366.304506  120.885096  12.036169  -3.433854 -24.169440 -13.973501   
10  -540.882321   49.557678  -2.491799  -7.023349  -6.225777   6.247800   
11  -393.523419   93.156385  23.004238   3.160594  -9.224882 -12.366414   
12  -251.518405  162.032558  12.211548   0.726063 -16.656684   3.548022   
13  -326.838738  144.327239   6.447565 -27.834843 -16.875479   7.490809   
14  -173.991198  134.575450 -42.192963  -6.033575 -26.256233  -1.971600   
15  -410.594556   79.353227   2.590714 -13.479014 -25.084634 -15.225662   
16  -162.174223  154.726445 -24.796252 -27.214147 -26.419858  -3.651262   
17  -399.709543   98.426396  12.833435  -7.143056 -18.696928 -17.757136   
18  -502.314108   23.699184 -30.881929   1.417022   2.171178   2.311240   
19  -371.814298  148.739859   1.139008  -6.799012   4.352360   5.029185   
20  -395.861763  120.598143 -10.012163 -41.370596 -44.266902 -19.220830   
21  -357.294547   74.903339   6.216953  -5.213464   0.948881  12.159967   
22  -325.441592  167.794607 -13.000734   9.387221 -11.614192   3.504295   
23  -439.369161   43.640000   8.440763  -3.924500 -11.537148  -8.508559   
24  -355.410548   78.624848 -80.062616 -35.142855 -33.235695 -29.348386   
25  -234.372586   99.578839 -82.462926 -36.530957 -20.341889   7.458714   
26  -444.499809  108.079066  -9.716749 -29.375695 -27.889706   2.131835   
27  -581.740034    8.206624  -6.658867  -4.291413  -3.034642   0.259200   
28  -405.505175   59.754379  -4.077395  -8.551207 -16.864026  -2.251054   
29  -420.419071  106.566067  15.553363 -14.629801 -20.385269 -16.875465   
..          ...         ...        ...        ...        ...        ...   
370 -300.094697   91.834682 -30.206425  38.955185  -1.753551  23.693829   
371 -118.555454  141.601235 -13.002415  68.341045   6.009578  27.019369   
372 -466.042948   95.505859  -7.380096  81.324246 -13.459093  30.590572   
373 -412.648476  130.760660  16.493260  69.380370  54.264565  42.981106   
374 -450.215165   78.638273  31.140161  64.243905   6.187755  36.104622   
375 -262.838798  110.003953  21.026210  52.077069  -8.068431  38.152427   
376 -422.213146  128.665364   2.531775  55.202192 -19.051791  35.370599   
377 -507.174153   22.817576  31.015973  15.710849   4.526292  14.061358   
378 -329.953256  100.782683   2.270215  56.558578 -13.734430  37.107715   
379 -288.395525   73.795755 -21.384930  60.607331 -11.472752  32.120504   
380 -195.614058   40.665138  15.420007  65.017738  22.672310  36.763715   
381 -355.792827   23.937104  14.717801  23.448826   6.587720   8.049350   
382 -405.908533  110.327779  15.449699  44.679646  25.794437  17.791511   
383 -523.785046   30.472092  -3.299369  76.284799  10.565552  39.687886   
384 -274.051274   76.631909 -18.934463  43.620297  -8.988119  40.505181   
385 -320.269833  121.692833  18.068000  52.281800 -12.309194  40.607972   
386 -305.585611  111.803554 -10.204981  53.519572  11.787420  38.870977   
387 -423.173648   70.572671  13.853737  59.451617   6.116873  43.431000   
388 -306.183278  106.442452  -6.884659  56.481412 -13.500079  38.301896   
389 -377.900565   92.241812   7.801299  38.116885  14.386035  32.430464   
390 -506.713039  101.703976 -19.196165  13.428451 -23.891664   3.358840   
391 -470.395324   42.142829  10.368376 

#### SVM Classification 

In [91]:
# coding= UTF-8
import numpy as np
import sklearn
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

#Load data from generated numpy files
X = np.load('feat.npy') # list of features
y = np.load('label.npy').ravel() # labels are the target

# Split into train and test sets (400 Audios total)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Data scaling (NOT IMPLEMENTING)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

# Implement simple linear SVM
svm_clf = SVC(C=28.0, gamma = 0.00001, decision_function_shape="ovr") #These parameters can be modified

# Fit model
svm_clf.fit(X_train, y_train) #From Beif github
#svm_clf.fit(X_train_scaled, y_train) # HandsOn book

# Make predictions
#y_pred = svm_clf.predict(X_train_scaled)
y_predict = svm_clf.predict(X_test)

#print('Prediction')
#print(y_predict)
#print
#print("Actual")
#print(y_test)

# Accuracy
acc = svm_clf.score(X_test, y_test)
print
print("accuracy=%0.3f" %acc)

Prediction
[3 2 0 4 6 1 3 6 3 5 5 3 2 8 0 1 7 3 5 4 4 6 3 5 4 1 1 0 2 2 4 2 8 5 1 6 2
 4 4 9 2 0 0 0 7 1 9 7 3 5 1 3 7 4 7 2 1 5 8 1 1 7 0 8 4 6 8 0 4 7 0 6 1 0
 2 0 9 7 9 8 8 1 1 1 7 5 3 9 9 8 9 6 1 8 4 3 3 4 7 9 6 1 7 3 1 8 2 8 3 7 2
 1 7 0 9 4 6 1 5 8 2 5 5 5 9 3 0 6 8 4 3 9 1 6 5 2 2 0 8 0 4 4 5 2 6 2 8 6
 5 2 8 3 6 2 5 3 4 3 9 7]

Actual
[3 7 8 4 6 1 3 6 3 5 0 3 2 8 0 1 7 3 5 4 4 6 3 8 4 7 1 0 7 2 4 7 8 5 1 1 2
 4 4 9 2 8 0 0 7 1 9 6 3 5 6 3 7 4 7 7 1 0 0 1 1 6 0 8 4 9 3 0 3 7 0 6 1 0
 2 0 9 7 9 8 8 1 1 2 7 5 3 9 9 8 9 6 1 0 4 3 3 9 6 9 6 1 7 3 7 8 2 8 3 7 2
 2 7 0 9 5 6 1 5 8 2 5 5 5 9 3 0 6 8 4 3 9 1 6 5 2 7 4 8 0 4 4 0 2 6 2 8 6
 5 2 8 3 7 2 5 3 6 3 4 7]

[ 0  5  8  0  0  0  0  0  0  0 -5  0  0  0  0  0  0  0  0  0  0  0  0  3
  0  6  0  0  5  0  0  5  0  0  0 -5  0  0  0  0  0  8  0  0  0  0  0 -1
  0  0  5  0  0  0  0  5  0 -5 -8  0  0 -1  0  0  0  3 -5  0 -1  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  0  0  0 -8  0  0
  0  5 -1  0  0  0  0  0  6  0  0  0

#### Accuracy results (Reporting Beif value)

- Without data scaling: 0.817 (C = 28)
- With data scaling: 0.375

#### CNN Classification (Keras Tensorflow)

In [8]:
# coding= UTF-8
import numpy as np
import keras 
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

# Load data 
X = np.load("feat.npy")
y = np.load('label.npy').ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 233)

# Neural Network Construction
model = Sequential()

# Architecture
model.add(Conv1D(64, 3, activation='relu', input_shape = (193, 1)))
model.add(Conv1D(64, 3, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(128, 3, activation='relu'))
model.add(Conv1D(128, 3, activation='relu'))
model.add(GlobalAveragePooling1D())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

# Which is the best loss function for binary (multiple) classification
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Convert label to onehot
y_train = keras.utils.to_categorical(y_train - 1, num_classes=10) # Converts a class vector (integers) to binary class matrix
y_test = keras.utils.to_categorical(y_test - 1, num_classes=10)

X_train = np.expand_dims(X_train, axis=2) # Make 2-dim into 3-dim array to fit model
X_test = np.expand_dims(X_test, axis=2)

# Train Network
model.fit(X_train, y_train, batch_size=64, epochs=100)

# Compute accuracy with test data
score, acc = model.evaluate(X_test, y_test, batch_size=16) # Computes the loss & accuracy based on the input you pass it

print('Test score:', score) #loss
print('Test accuracy:', acc)

Epoch 1/100
240/240 [==============================] - 2s 9ms/step - loss: 2.3719 - acc: 0.0958
Epoch 2/100
240/240 [==============================] - 1s 6ms/step - loss: 2.2430 - acc: 0.1708
Epoch 3/100
240/240 [==============================] - 1s 6ms/step - loss: 2.0834 - acc: 0.2375
Epoch 4/100
240/240 [==============================] - 2s 7ms/step - loss: 2.0294 - acc: 0.3375
Epoch 5/100
240/240 [==============================] - 2s 7ms/step - loss: 1.9173 - acc: 0.3000
Epoch 6/100
240/240 [==============================] - 2s 7ms/step - loss: 1.7578 - acc: 0.3792
Epoch 7/100
240/240 [==============================] - 2s 8ms/step - loss: 1.7275 - acc: 0.3875
Epoch 8/100
240/240 [==============================] - 2s 7ms/step - loss: 1.6114 - acc: 0.3792
Epoch 9/100
240/240 [==============================] - 2s 8ms/step - loss: 1.6410 - acc: 0.4083
Epoch 10/100
240/240 [==============================] - 3s 11ms/step - loss: 1.5011 - acc: 0.4292
Epoch 11/100
240/240 [================

In [2]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(240, 193, 1)
(160, 193, 1)
(240, 10)
(160, 10)


- Common Keras NN workflow: Sequential, Add,Compile, Fit
- Accuracy with CNN: 0.78125 (1000 epochs)
- Accuracy: 0.7125 (100 epochs)

#### Multilayer Perceptron Classification (Keras Tensorflow)

In [1]:
# coding= UTF-8
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split

# Prepare the data
X =  np.load('feat.npy')
y =  np.load('label.npy').ravel() #Return a contiguous flattened array.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Build the Neural Network
model = Sequential()
model.add(Dense(512, activation='relu', input_dim=193)) ## Dense method for MLP
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Convert label to onehot
y_train = keras.utils.to_categorical(y_train-1, num_classes=10) # Convert class vector into binary Matrix
y_test = keras.utils.to_categorical(y_test-1, num_classes=10)

# Train and test
model.fit(X_train, y_train, epochs=100, batch_size=64) # Epochs are tunable
score, acc = model.evaluate(X_test, y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

/home/hackerman/anaconda2/envs/anaconda2_py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Epoch 1/100
240/240 [==============================] - 0s 453us/step - loss: 13.4215 - acc: 0.1125
Epoch 2/100
240/240 [==============================] - 0s 349us/step - loss: 13.3288 - acc: 0.1458
Epoch 3/100
240/240 [==============================] - 0s 350us/step - loss: 11.8501 - acc: 0.2167
Epoch 4/100
240/240 [==============================] - 0s 345us/step - loss: 11.6624 - acc: 0.2125
Epoch 5/100
240/240 [==============================] - 0s 298us/step - loss: 11.1756 - acc: 0.2625
Epoch 6/100
240/240 [==============================] - 0s 200us/step - loss: 10.8737 - acc: 0.2792
Epoch 7/100
240/240 [==============================] - 0s 325us/step - loss: 11.5427 - acc: 0.2500
Epoch 8/100
240/240 [==============================] - 0s 457us/step - loss: 11.3938 - acc: 0.2708
Epoch 9/100
240/240 [==============================] - 0s 441us/step - loss: 11.4216 - acc: 0.2583
Epoch 10/100
240/240 [==============================] - 0s 445us/step - loss: 10.2797 - acc: 0.3167
Epoch 11/

- Accuracy with MLP: 0.80625 (250 epochs)
- Accuracy: 0.63125 (100 epochs)
- MLP is the faster of the 3 neural networks
- A classification predictor can be visualized by drawing the boundary line.

#### Naive Bayes (NB) Classification (Sklearn)

In [3]:
# coding= UTF-8
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import pandas as pd 

#Load data 
X = np.load('feat.npy') 
y = np.load('label.npy').ravel() 

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# Initialize classifier
gnb_clf= GaussianNB() #check input params

# Train model
gnb_clf.fit(X_train, y_train)
#model = gnb_clf.fit(X_train, y_train)

# Make predictions
prediction = gnb_clf.predict(X_test)

#print('Predicted values')
#print(prediction)
#print
#print('Actual values')
#print(y_test)
#print

# Evaluate accuracy
#Similar ways to do it
#print(accuracy_score(y_test,prediction)) 
print
acc = gnb_clf.score(X_test, y_test) 
print("Accuracy = %0.3f" %acc)

Predicted values
[6 2 0 9 5 9 8 2 7 3 0 4 1 1 7 1 6 4 8 9 1 0 9 5 8 4 2 5 1 2 9 4 6 2 5 2 2
 8 0 3 4 1 6 3 2 0 3 4 0 4 9 6 8 1 4 0 5 4 8 3 3 6 7 7 2 0 8 6 9 1 8 3 0 1
 9 6 5 9 4 7 0 3 0 6 1 1 6 7 2 2 5 8 0 5 4 6 9 9 5 1 6 2 7 1 3 0 0 3 8 8 0
 4 7 9 7 1 9 7 7 1 0 2 0 8 0 0 8 8 4 6 6 3]

Actual values
[5 7 0 5 2 2 8 2 6 3 0 9 1 1 3 1 6 9 5 9 1 0 9 6 0 9 2 5 6 2 9 4 6 1 5 2 2
 8 3 3 9 1 1 0 2 9 3 4 0 4 1 8 8 1 9 0 5 4 0 3 3 7 6 1 2 0 8 6 9 1 8 3 0 1
 9 6 5 9 4 7 0 0 5 6 6 1 6 7 2 2 5 8 2 5 4 7 9 9 5 1 6 2 0 1 3 0 0 3 8 8 3
 9 7 9 7 1 1 7 7 7 3 2 0 8 0 4 8 8 4 6 7 3]

[ 1 -5  0  4  3  7  0  0  1  0  0 -5  0  0  4  0  0 -5  3  0  0  0  0 -1
  8 -5  0  0 -5  0  0  0  0  1  0  0  0  0 -3  0 -5  0  5  3  0 -9  0  0
  0  0  8 -2  0  0 -5  0  0  0  8  0  0 -1  1  6  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  3 -5  0 -5  0  0  0  0  0  0  0 -2  0  0 -1
  0  0  0  0  0  0  7  0  0  0  0  0  0  0 -3 -5  0  0  0  0  8  0  0 -6
 -3  0  0  0  0 -4  0  0  0  0 -1  0]

Accuracy = 0.697


- Accuracy with NB: 0.697 (33% Test data)

#### Random Forest Classification (Sklearn)

In [4]:
# coding= UTF-8
from sklearn.ensemble import RandomForestClassifier #Random Forest classifier
import pandas as pd 
import numpy as np
np.random.seed(0)

In [5]:
#Load data 
X = np.load('feat.npy') 
y = np.load('label.npy').ravel() 

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

# Initialize classifier
rf_clf = RandomForestClassifier(n_jobs=2, random_state=0) #Check params

# Train model
rf_clf.fit(X_train, y_train)

# Make predictions
y_prediction = rf_clf.predict(X_test)

#print('Predicted values')
#print(y_prediction)
#print
#print('Actual values')
#print(y_test)
#print

# Evaluate accuracy
print
acc = rf_clf.score(X_test, y_test) 
print("Accuracy = %0.3f" %acc)

Predicted values
[5 7 0 9 7 1 8 2 6 3 4 9 1 1 7 1 6 9 3 9 2 0 9 0 0 9 2 5 6 2 9 4 6 1 5 2 2
 8 3 3 9 1 6 0 2 9 3 4 3 4 1 6 8 1 9 0 5 4 0 3 3 7 7 2 2 0 8 6 9 1 0 3 0 1
 9 6 5 9 4 7 0 4 0 6 1 1 6 7 2 2 5 4 2 5 4 2 4 9 1 1 6 2 7 1 3 0 0 3 3 8 3
 9 7 9 7 1 1 7 7 2]

Actual values
[5 7 0 5 2 2 8 2 6 3 0 9 1 1 3 1 6 9 5 9 1 0 9 6 0 9 2 5 6 2 9 4 6 1 5 2 2
 8 3 3 9 1 1 0 2 9 3 4 0 4 1 8 8 1 9 0 5 4 0 3 3 7 6 1 2 0 8 6 9 1 8 3 0 1
 9 6 5 9 4 7 0 0 5 6 6 1 6 7 2 2 5 8 2 5 4 7 9 9 5 1 6 2 0 1 3 0 0 3 8 8 3
 9 7 9 7 1 1 7 7 7]

[ 0  0  0  4  5 -1  0  0  0  0  4  0  0  0  4  0  0  0 -2  0  1  0  0 -6
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  5  0  0  0  0  0
  3  0  0 -2  0  0  0  0  0  0  0  0  0  0  1  1  0  0  0  0  0  0 -8  0
  0  0  0  0  0  0  0  0  0  4 -5  0 -5  0  0  0  0  0  0 -4  0  0  0 -5
 -5  0 -4  0  0  0  7  0  0  0  0  0 -5  0  0  0  0  0  0  0  0  0  0 -5]

Accuracy = 0.800


- Accuracy with Random Forest: 0.800 (30% test data)

In [6]:
# View the predicted probabilities of the first n observations
rf_clf.predict_proba(X_test)[0:10]

array([[0. , 0. , 0.1, 0. , 0. , 0.8, 0.1, 0. , 0. , 0. ],
       [0. , 0. , 0.2, 0. , 0. , 0. , 0. , 0.8, 0. , 0. ],
       [0.7, 0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0.1, 0. ],
       [0. , 0.1, 0. , 0.1, 0.3, 0.1, 0. , 0. , 0. , 0.4],
       [0. , 0.1, 0.2, 0. , 0. , 0.2, 0.2, 0.3, 0. , 0. ],
       [0. , 0.3, 0.3, 0. , 0. , 0. , 0.3, 0.1, 0. , 0. ],
       [0. , 0. , 0. , 0.1, 0.1, 0. , 0. , 0. , 0.8, 0. ],
       [0. , 0.1, 0.8, 0. , 0. , 0.1, 0. , 0. , 0. , 0. ],
       [0. , 0.1, 0.1, 0. , 0. , 0.1, 0.4, 0.2, 0. , 0.1],
       [0. , 0. , 0. , 1. , 0. , 0. , 0. , 0. , 0. , 0. ]])

In [9]:
# Dencoding predicted and actual classes (numeric to written)
prediction_decoded = label_classes[y_prediction]
actual_value_decoded = label_classes[y_test]
#print(y_prediction)
#print(y_test)
print('Prediction decoded')
print(prediction_decoded)
print
print('Actual class decoded')
print(actual_value_decoded)

Prediction decoded
['Person sneeze' 'Chainsaw' 'Dog bark' 'Fire crackling' 'Chainsaw' 'Rain'
 'Rooster' 'Sea waves' 'Helicopter' 'Baby cry' 'Clock tick'
 'Fire crackling' 'Rain' 'Rain' 'Chainsaw' 'Rain' 'Helicopter'
 'Fire crackling' 'Baby cry' 'Fire crackling' 'Sea waves' 'Dog bark'
 'Fire crackling' 'Dog bark' 'Dog bark' 'Fire crackling' 'Sea waves'
 'Person sneeze' 'Helicopter' 'Sea waves' 'Fire crackling' 'Clock tick'
 'Helicopter' 'Rain' 'Person sneeze' 'Sea waves' 'Sea waves' 'Rooster'
 'Baby cry' 'Baby cry' 'Fire crackling' 'Rain' 'Helicopter' 'Dog bark'
 'Sea waves' 'Fire crackling' 'Baby cry' 'Clock tick' 'Baby cry'
 'Clock tick' 'Rain' 'Helicopter' 'Rooster' 'Rain' 'Fire crackling'
 'Dog bark' 'Person sneeze' 'Clock tick' 'Dog bark' 'Baby cry' 'Baby cry'
 'Chainsaw' 'Chainsaw' 'Sea waves' 'Sea waves' 'Dog bark' 'Rooster'
 'Helicopter' 'Fire crackling' 'Rain' 'Dog bark' 'Baby cry' 'Dog bark'
 'Rain' 'Fire crackling' 'Helicopter' 'Person sneeze' 'Fire crackling'
 'Clock tick' '

In [10]:
## Confusion Matrix
pd.crosstab(actual_value_decoded, prediction_decoded)
#pd.crosstab(test['species'], preds, rownames=['Actual Species'], colnames=['Predicted Species'])

col_0           Baby cry  Chainsaw  Clock tick  Dog bark  Fire crackling  \
row_0                                                                      
Baby cry              10         1           0         0               0   
Chainsaw               0         8           0         0               0   
Clock tick             0         0           6         0               0   
Dog bark               1         1           2        11               0   
Fire crackling         0         0           1         0              15   
Helicopter             0         1           0         1               0   
Person sneeze          1         0           0         1               1   
Rain                   0         0           0         0               0   
Rooster                1         0           1         1               0   
Sea waves              0         1           0         0               0   

col_0           Helicopter  Person sneeze  Rain  Rooster  Sea waves  
row_0                                                                
Baby cry                 0              0     0        0          0  
Chainsaw                 0              0     0        0          2  
Clock tick               0              0     0        0          0  
Dog bark                 0              0     0        0          0  
Fire crackling           0              0     0        0          0  
Helicopter               9              0     1        0          0  
Person sneeze            0              7     1        0          0  
Rain                     1              0    14        0          2  
Rooster                  1              0     0        5          0  
Sea waves                0              0     1        0         11

#### RNN Classification (Keras Tensorflow)

- Sigmoid vs Softmax: The sigmoid function is used for the two-class logistic regression (0 or 1, speech or non-speech), whereas the softmax function is used for the multiclass logistic regression (a.k.a. MaxEnt, multinomial logistic regression, softmax Regression, Maximum Entropy Classifier). (dog bark, sea waves, ...)
- Network Architecture: Regarding more general choices, there is rarely a "right" way to construct the architecture. Instead that should be something you test with different meta-params (such as layer sizes, number of layers, amount of drop-out), and should be results-driven (including any limits you might have on resource use for training time/memory use etc).
- https://datascience.stackexchange.com/questions/10048/what-is-the-best-keras-model-for-multi-class-classification

In [1]:
# coding= UTF-8
import os
import numpy as np
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense
from keras.optimizers import Adam

# Load data 
X = np.load("feat.npy")
y = np.load('label.npy').ravel()

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 233)

#batch_size = 35
# nb_epochs = 400

# Reshape data for LSTM (Samples, Timesteps, Features)
X_train = np.expand_dims(X_train, axis=2) #(280,193,1)
X_test = np.expand_dims(X_test, axis=2)

y_train = keras.utils.to_categorical(y_train - 1, num_classes=10) # Converts a class vector (integers) to binary class matrix
y_test = keras.utils.to_categorical(y_test - 1, num_classes=10)

# Build RNN Neural Network
print('Build LSTM RNN model ...')
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=X_train.shape[1:]))
model.add(LSTM(32, return_sequences=False))
model.add(Dense(y_train.shape[1], activation='softmax'))
#model.add(Dense(10, activation='sigmoid'))

#model.add(LSTM(units=128, dropout=0.05, recurrent_dropout=0.35, return_sequences=True, input_shape=input_shape))
#model.add(LSTM(units=32, dropout=0.05, recurrent_dropout=0.35, return_sequences=False))
#model.add(Dense(units=genre_features.train_Y.shape[1], activation='softmax'))
          
print("Compiling ...")
model.compile(loss='categorical_crossentropy', # for multiple classes
              optimizer='adam', 
              metrics=['accuracy'])

print(model.summary())

print("Training ...")
model.fit(X_train, y_train, batch_size=35, epochs=100)

print("\nValidating ...")
score, accuracy = model.evaluate(X_test, y_test, batch_size=35, verbose=1)
print("Loss:  ", score)
print("Accuracy:  ", accuracy)

/home/hackerman/anaconda2/envs/anaconda2_py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Build LSTM RNN model ...
Compiling ...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 193, 128)          66560     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                20608     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 87,498
Trainable params: 87,498
Non-trainable params: 0
_________________________________________________________________
None
Training ...
Epoch 1/100
300/300 [==============================] - 9s 30ms/step - loss: 2.3073 - acc: 0.1300
Epoch 2/100
300/300 [==============================] - 9s 30ms/step - loss: 2.2820 - acc: 0.0933
Epoch 3/100
300/300 [==============================] - 10s 32ms/step - loss: 2.2082 - acc: 0.2167
Epoch 4/100
300/300 [==============================] 

- RNN Accuracy: 0.660 (25% Test data, 100 epochs)

#### Overall Accuracy Results 


In [2]:
import pandas as pd

df = pd.DataFrame({'SVM': [81.7, 0,],
                   'CNN': [78.125, 71.25],
                   'MLP':[80.625, 63.125],
                   'NB': [69.7, 0],
                   'RF': [80, 0],
                   'RNN': [66, 0],})
df

CNN     MLP    NB  RF  RNN   SVM
0  78.125  80.625  69.7  80   66  81.7
1  71.250  63.125   0.0   0    0   0.0